In [19]:
import nltk
import pandas as pd
import numpy as np
import csv
data = pd.read_csv(r"D:\SLU\AI MSc\Fall 22\NLP\Project 2\train.tsv", sep="\t", header=None, quoting = csv.QUOTE_NONE)
data.rename(columns={0:"Word", 1:"Label"}, inplace=True)

In [20]:
data.head()

,Word,Label
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N


In [21]:
data[data['Label'] == 'U']

,Word,Label
14,cearta,U
43,bíonn,U
258,bóthar,U
287,éiríonn,U
424,cuirfidh,U
...,...,...
9999956,bun,U
9999958,imscrúduithe,U
9999965,formhór,U
9999984,feidhm,U


In [22]:
# Covert dataframe to list of tuples

words = [(data.loc[i, 'Word'], data.loc[i, 'Label']) for i in range(len(data))]

In [23]:
# find the frequency of words

fd = nltk.FreqDist(tag for (word,tag) in words)
fd

FreqDist({'N': 8584022, 'S': 973469, 'U': 327110, 'H': 80504, 'T': 34895})

In [24]:
# conditional freqency

cfd = nltk.ConditionalFreqDist(words)
print(cfd['cearta']['U'])
print(cfd['cearta']['S'])
print(cfd['cearta']['N'])

137
723
1379


In [26]:
words[0:3]

[('ansin', 'N'), (')', 'N'), ('tá', 'N')]

In [38]:
# list of just words

words_list = data['Word'].values.tolist()
words_list

['ansin',
 ')',
 'tá',
 'níos',
 'lú',
 'gaeilge',
 'ag',
 'na',
 'gardaí',
 'ná',
 'bí',
 'ariamh',
 'ainneoin',
 'na',
 'cearta',
 '.',
 'níl',
 'sé',
 'ach',
 'roinnt',
 'seachtainí',
 'ó',
 'sin',
 'a',
 'tógadh',
 'fear',
 'bocht',
 'a',
 'tug',
 'ainm',
 'gaeilge',
 'dóibh',
 '.',
 '<S>',
 'socraíodh',
 'go',
 'raibh',
 'gá',
 'lena',
 'leithéid',
 ',',
 'mar',
 'go',
 'bíonn',
 'na',
 '<S>',
 'tá',
 'an',
 'córas',
 'bainistíochta',
 'tar',
 'éis',
 'freastal',
 'go',
 'maith',
 'ar',
 'rialtas',
 'áitiúil',
 'na',
 'éireann',
 'agus',
 'leanfaidh',
 'údaráis',
 'áitiúla',
 'ar',
 'aghaidh',
 'ag',
 'brath',
 'ar',
 'bainisteoirí',
 'gairmiúla',
 'le',
 'treoir',
 'agus',
 'riarachán',
 'gairmiúil',
 'neamhchlaon',
 'a',
 'soláthar',
 '.',
 '<S>',
 'john',
 "o'donovanba",
 'luath',
 'a',
 "d'éirigh",
 'larcom',
 'as',
 'a',
 'beith',
 'ag',
 'dul',
 'don',
 'gaeilge',
 'agus',
 'a',
 'roghnaigh',
 'réiteach',
 'eile',
 'áfach',
 '–',
 'fostú',
 'scoláirí',
 'gaeilge',
 'nó',
 '‘

In [40]:
justwords = nltk.FreqDist(words_list).keys()

In [ ]:
justwords

In [42]:
best_tags = dict((w, cfd[w].max()) for w in justwords)
print('Most frequent tag for "the":', best_tags['cearta'])
print('Most frequent tag for "pay":', best_tags['tír'])

Most frequent tag for "the": N
Most frequent tag for "pay": N


In [49]:
split = int(len(words)*0.9)
train = [words[:split]]
test = [words[split:]]
unigram_tagger = nltk.UnigramTagger(train)
unigram_tagger.accuracy(test)

In [ ]:
flipped = [(t,w) for sent in train for (w,t) in sent]
wordgiventag = nltk.ConditionalFreqDist(flipped)

In [ ]:
# this is P(w|t), unsmoothed!
def P(w,t):
    return wordgiventag[t][w] / wordgiventag[t].N()

'''print('P(the|DET) =', P('the','DET'))
print('P(is|VERB) =', P('is','VERB'))
print('P(pay|VERB) =', P('pay','VERB'))'''

In [ ]:
tag_bigrams = [(x,y) for sent in train for x,y in nltk.bigrams([t for (w,t) in sent])]

In [ ]:
tag_bigram_counts = nltk.ConditionalFreqDist(tag_bigrams)
# this is count of noun tags following adjective tags (normal order in English)
'''print(tag_bigram_counts['ADJ']['NOUN'])
# this is count of adjective tags following noun tags
print(tag_bigram_counts['NOUN']['ADJ'])'''

In [ ]:
# this is P(t2|t1), unsmoothed again!
def tagP(t2,t1):
    return tag_bigram_counts[t1][t2] / tag_bigram_counts[t1].N()
'''print('P(NOUN|ADJ) =',tagP('NOUN','ADJ'))
print('P(NOUN|DET) =',tagP('NOUN','DET'))'''

In [ ]:
sentence_start = nltk.FreqDist(sent[0][1] for sent in train)
def initP(t):
    return sentence_start[t] / sentence_start.N()
'''print('initP(DET) =', initP('DET'))
print('initP(PRON) =', initP('PRON'))  # he, she, it, etc.
print('initP(NOUN) =', initP('NOUN'))
print('initP(VERB) =', initP('VERB'))'''

In [ ]:
def argmax(V,tag_list,t,i):
    ans=-1
    best=None
    for s in tag_list:
        temp=V[(s,i-1)]*tagP(t,s)
        if temp > ans:
            ans = temp
            best = s
    return (best,ans)

In [ ]:
def printV(sentence,tag_list,V,B):
    for i in range(len(sentence)):
        print('i='+str(i)+' ['+sentence[i]+']')
        for t in tag_list:
            if V[(t,i)] != 0:
                toprint='  '+t+'='+str(V[(t,i)])
                if i>0:
                    toprint += ' (from '+B[(t,i)]+')'
                print(toprint)
    

In [ ]:
def viterbi(sentence):
    V = dict()    # keys are (t,i) where t is a tag (row label) and i is position in sentence (column label)
    B = dict()    # same keys as V; this stores the "backpointers" to remember best tag sequence
    tag_list = sentence_start.keys()
    for t in tag_list:
        V[(t,0)] = initP(t)*P(sentence[0],t)
    for i in range(1,len(sentence)):
        for t in tag_list:
            pair = argmax(V,tag_list,t,i)
            B[(t,i)] = pair[0]
            V[(t,i)] = pair[1]*P(sentence[i],t)
    printV(sentence,tag_list,V,B)